In [1]:
# For compatibility across multiple platforms
import os
import numpy as np
import pandas as pd
from scipy import spatial


# Load files using DictReader in Python
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import cluster
from sklearn import preprocessing
from sklearn.model_selection import KFold
import sklearn

In [2]:
train_filepath = "pubg-finish-placement-prediction/train_V2_clean.csv"

#trainset_file = open(train_filepath,'rU')
trainset = pd.read_csv(train_filepath,index_col=0)


C:\Users\zacfa\.julia\conda\3\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
print(len(trainset))

4446965


## Remove Illegal Match

In [4]:
trainset = trainset.drop(["matchType"],axis=1)

# Normalization and feature engineering

Add a feature for the number of players joined

In [6]:
trainset['playersJoined'] = trainset.groupby('matchId')['matchId'].transform('count')
trainset['killsNorm'] = trainset['kills']*((100-trainset['playersJoined'])/100 + 1)
trainset['damageDealtNorm'] = trainset['damageDealt']*((100-trainset['playersJoined'])/100 + 1)
trainset['maxPlaceNorm'] = trainset['maxPlace']*((100-trainset['playersJoined'])/100 + 1)
trainset['matchDurationNorm'] = trainset['matchDuration']*((100-trainset['playersJoined'])/100 + 1)

# Train on XGBoost

In [7]:
train_columns = list(trainset.columns[:-1])
x = trainset[train_columns]
y = trainset["winPlacePerc"]


x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

In [8]:
x_train.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc,playersJoined,killsNorm,damageDealtNorm,maxPlaceNorm
1981117,5887056756028409,35383659749828605,13492780636643902,0,0,251.50,2,0,0,35,...,0,0,111.0,2,1656,0.1852,92,1.08,271.6200,30.24
2068597,22097650090541178,21717888397608285,68678329019572552,0,2,320.00,4,0,2,20,...,0,0,959.6,7,0,0.4074,96,2.08,332.8000,29.12
397627,63190309067454694,60884390273842220,63255699697520731,0,1,0.00,0,0,0,59,...,0,0,430.0,5,0,0.5000,98,0.00,0.0000,27.54
1947783,2978679697640915,24549235125563997,55663622350125375,0,0,95.16,0,0,0,42,...,0,0,2024.0,9,0,0.8667,91,0.00,103.7244,50.14
460933,34601890870789031,67509994436052565,43144151795316721,0,0,0.00,0,0,0,58,...,0,0,2258.0,7,1510,0.6333,100,0.00,0.0000,31.00


In [9]:
print(trainset.columns)

Index(['Id', 'groupId', 'matchId', 'assists', 'boosts', 'damageDealt', 'DBNOs',
       'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration', 'maxPlace', 'numGroups',
       'rankPoints', 'revives', 'rideDistance', 'roadKills', 'swimDistance',
       'teamKills', 'vehicleDestroys', 'walkDistance', 'weaponsAcquired',
       'winPoints', 'winPlacePerc', 'playersJoined', 'killsNorm',
       'damageDealtNorm', 'maxPlaceNorm', 'matchDurationNorm'],
      dtype='object')


In [10]:
trainset_dmatrix = xgb.DMatrix(x_train.values,label=y_train.values)
valset_dmatrix = xgb.DMatrix(x_val.values,label=y_val.values)

In [11]:
params = {
    "max_depth" : 10,
    "eval_metric" : ["mae"],
}

In [ ]:
clf = xgb.train(params, trainset_dmatrix, evals=[(trainset_dmatrix, "train"),(valset_dmatrix, 'val')], num_boost_round = 100)

predictions = xgb.predict(valset_dmatrix)

[13:34:24] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[0]	train-mae:0.188647	val-mae:0.188373
[13:34:31] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[1]	train-mae:0.132134	val-mae:0.131942
[13:34:38] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[2]	train-mae:0.0925	val-mae:0.092365
[13:34:45] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[3]	train-mae:0.064759	val-mae:0.064666
[13:34:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[4]	train-mae:0.045336	val-mae:0.04527
[13:35:00] C:\Users\Administrator\Desktop\xgboos